In [1]:
import numpy as np

import jwst
from jwst.pipeline import Image3Pipeline
from jwst.datamodels import ImageModel, ModelContainer

import logging
from jwst.resample import resample_step
resample_step.log.setLevel(logging.DEBUG)

In [2]:
skystat = 'median'  # 'mean' or 'mode'
kernel = 'square'
weight_type = 'exptime'

In [3]:
steps = {
    'assign_mtwcs': {
        'output_file': None, 'output_dir': None,
        'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True,
        'save_results': False, 'skip': False, 'suffix': 'assign_mtwcs',
        'search_output_file': True, 'input_dir': ''
    },
    'tweakreg': {
        'output_file': None, 'output_dir': None,
        'output_ext': '.fits', 'output_use_model': True, 'output_use_index': True,
        'save_results': False, 'skip': False, 'suffix': None, 'search_output_file': True,
        'input_dir': '', 'save_catalogs': True, 'use_custom_catalogs': False,
        'catalog_format': 'ecsv', 'catfile': '', 'kernel_fwhm': 2.165, 
        'sharplo': 0.3, 'sharphi': 0.9, 'roundlo': -0.25, 'roundhi': 0.25,
        'peakmax': None, 'bkg_boxsize': 400, 
        'use2dhist': True, 
        'xoffset': 0.0, 'yoffset': 0.0,

        'snr_threshold': 100,
        'brightest': 300,
        'minobj': 5,
        'searchrad': 40,
        'separation': 1,
        'tolerance': 0.05,
        'enforce_user_order': True,
        'expand_refcat': True,
        'fitgeometry': 'shift', # OR rshift
        'nclip': 5, 'sigma': 2.5,
        
        #'abs_refcat': '/orange/adamginsburg/jwst/cloudc//F405N/pipeline/jw02221-o002_t001_nircam_clear-F405N-merged_vvvcat.ecsv',
        'abs_refcat': '',
        'save_abs_catalog': False, 'abs_minobj': 15, 'abs_searchrad': 1, 'abs_use2dhist': True,
        'abs_separation': 0.1, 'abs_tolerance': 0.7, 'abs_fitgeometry': 'rshift', 'abs_nclip': 3,
        'abs_sigma': 3.0
    },
    'skymatch': {
        'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False,
        'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None,
        'search_output_file': True, 'input_dir': '', 'skymethod': 'match', 'match_down': True,
        'subtract': False, 'stepsize': None, 'skystat': 'mode', 'dqbits': '~DO_NOT_USE+NON_SCIENCE',
        'lower': None, 'upper': None, 'nclip': 5, 'lsigma': 4.0, 'usigma': 4.0, 'binwidth': 0.1
    },
    'outlier_detection': {
        'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False,
        'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None,
        'search_output_file': False, 'input_dir': '', 'weight_type': 'ivm', 'pixfrac': 1.0,
        'kernel': 'square', 'fillval': 'INDEF', 'nlow': 0, 'nhigh': 0, 'maskpt': 0.7, 'grow': 1,
        'snr': '5.0 4.0', 'scale': '1.2 0.7', 'backg': 0.0, 'kernel_size': '7 7',
        'threshold_percent': 99.8, 'ifu_second_check': False, 'save_intermediate_results': False,
        'resample_data': True, 'good_bits': '~DO_NOT_USE', 'scale_detection': False,
        'allowed_memory': None, 'in_memory': False
    },
    'resample': {
        'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False,
        'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': None,
        'search_output_file': True, 'input_dir': '', 'pixfrac': 1.0, 'kernel': 'square',
        'fillval': 'INDEF', 'weight_type': 'ivm', 'crpix': None, 'crval': None,
        'rotation': None, 'pixel_scale_ratio': 1.0, 'pixel_scale': None, 'output_wcs': 'ref_wcs.asdf',
        'single': False, 'blendheaders': True, 'allowed_memory': None, 'in_memory': True,
        'output_shape': None, 'output_wcs': '',
    },
    'source_catalog': {
        'output_file': None, 'output_dir': None, 'output_ext': '.fits', 'output_use_model': False,
        'output_use_index': True, 'save_results': False, 'skip': False, 'suffix': 'cat',
        'search_output_file': True, 'input_dir': '', 'bkg_boxsize': 1000, 'kernel_fwhm': 2.0,
        'snr_threshold': 3.0, 'npixels': 25, 'deblend': False, 'aperture_ee1': 30, 'aperture_ee2': 50,
        'aperture_ee3': 70, 'ci1_star_threshold': 2.0, 'ci2_star_threshold': 1.8
    }
}

In [4]:
mc = ModelContainer('jw02221-o002_20230911t112751_image3_00006_asn.json')

In [5]:
custom_order = [
    'jw02221002001_02201_00001_nrca2_cal.fits',
    'jw02221002001_02201_00002_nrca2_cal.fits',
    'jw02221002001_02201_00003_nrca2_cal.fits',
    'jw02221002001_02201_00004_nrca2_cal.fits',
    'jw02221002001_08201_00001_nrca4_cal.fits',
    'jw02221002001_08201_00002_nrca4_cal.fits',
    'jw02221002001_08201_00003_nrca4_cal.fits',
    'jw02221002001_08201_00004_nrca4_cal.fits',
    'jw02221002002_08201_00001_nrcb3_cal.fits',
    'jw02221002002_08201_00002_nrcb3_cal.fits',
    'jw02221002002_08201_00003_nrcb3_cal.fits',
    'jw02221002002_08201_00004_nrcb3_cal.fits',
    'jw02221002002_02201_00001_nrcb3_cal.fits',
    'jw02221002002_02201_00002_nrcb3_cal.fits',
    'jw02221002002_02201_00003_nrcb3_cal.fits',
    'jw02221002002_02201_00004_nrcb3_cal.fits',
    'jw02221002002_08201_00001_nrcb1_cal.fits',
    'jw02221002002_08201_00002_nrcb1_cal.fits',
    'jw02221002002_08201_00003_nrcb1_cal.fits',
    'jw02221002002_08201_00004_nrcb1_cal.fits',
    'jw02221002001_02201_00001_nrca4_cal.fits',
    'jw02221002001_02201_00002_nrca4_cal.fits',
    'jw02221002001_02201_00003_nrca4_cal.fits',
    'jw02221002001_02201_00004_nrca4_cal.fits',
    'jw02221002001_08201_00001_nrca2_cal.fits',
    'jw02221002001_08201_00002_nrca2_cal.fits',
    'jw02221002001_08201_00003_nrca2_cal.fits',
    'jw02221002001_08201_00004_nrca2_cal.fits',
    'jw02221002002_02201_00001_nrca4_cal.fits',
    'jw02221002002_02201_00002_nrca4_cal.fits',
    'jw02221002002_02201_00003_nrca4_cal.fits',
    'jw02221002002_02201_00004_nrca4_cal.fits',
    'jw02221002001_02201_00001_nrca1_cal.fits',
    'jw02221002001_02201_00002_nrca1_cal.fits',
    'jw02221002001_02201_00003_nrca1_cal.fits',
    'jw02221002001_02201_00004_nrca1_cal.fits',
    'jw02221002001_08201_00001_nrca3_cal.fits',
    'jw02221002001_08201_00002_nrca3_cal.fits',
    'jw02221002001_08201_00003_nrca3_cal.fits',
    'jw02221002001_08201_00004_nrca3_cal.fits',
    'jw02221002002_02201_00001_nrcb4_cal.fits',
    'jw02221002002_02201_00002_nrcb4_cal.fits',
    'jw02221002002_02201_00003_nrcb4_cal.fits',
    'jw02221002002_02201_00004_nrcb4_cal.fits',
    'jw02221002002_08201_00001_nrcb2_cal.fits',
    'jw02221002002_08201_00002_nrcb2_cal.fits',
    'jw02221002002_08201_00003_nrcb2_cal.fits',
    'jw02221002002_08201_00004_nrcb2_cal.fits',
    'jw02221002002_08201_00001_nrcb4_cal.fits',
    'jw02221002002_08201_00002_nrcb4_cal.fits',
    'jw02221002002_08201_00003_nrcb4_cal.fits',
    'jw02221002002_08201_00004_nrcb4_cal.fits',
    'jw02221002001_02201_00001_nrca3_cal.fits',
    'jw02221002001_02201_00002_nrca3_cal.fits',
    'jw02221002001_02201_00003_nrca3_cal.fits',
    'jw02221002001_02201_00004_nrca3_cal.fits',
    'jw02221002001_08201_00001_nrcb3_cal.fits',
    'jw02221002001_08201_00002_nrcb3_cal.fits',
    'jw02221002001_08201_00003_nrcb3_cal.fits',
    'jw02221002001_08201_00004_nrcb3_cal.fits',
    'jw02221002002_02201_00001_nrcb1_cal.fits',
    'jw02221002002_02201_00002_nrcb1_cal.fits',
    'jw02221002002_02201_00003_nrcb1_cal.fits',
    'jw02221002002_02201_00004_nrcb1_cal.fits',
    'jw02221002001_08201_00001_nrcb4_cal.fits',
    'jw02221002001_08201_00002_nrcb4_cal.fits',
    'jw02221002001_08201_00003_nrcb4_cal.fits',
    'jw02221002001_08201_00004_nrcb4_cal.fits',
    'jw02221002002_02201_00001_nrcb2_cal.fits',
    'jw02221002002_02201_00002_nrcb2_cal.fits',
    'jw02221002002_02201_00003_nrcb2_cal.fits',
    'jw02221002002_02201_00004_nrcb2_cal.fits',
    'jw02221002002_02201_00001_nrca3_cal.fits',
    'jw02221002002_02201_00002_nrca3_cal.fits',
    'jw02221002002_02201_00003_nrca3_cal.fits',
    'jw02221002002_02201_00004_nrca3_cal.fits',
    'jw02221002001_08201_00001_nrca1_cal.fits',
    'jw02221002001_08201_00002_nrca1_cal.fits',
    'jw02221002001_08201_00003_nrca1_cal.fits',
    'jw02221002001_08201_00004_nrca1_cal.fits',
    
    'jw02221002002_08201_00001_nrca3_cal.fits',
    'jw02221002002_08201_00002_nrca3_cal.fits',
    'jw02221002002_08201_00003_nrca3_cal.fits',
    'jw02221002002_08201_00004_nrca3_cal.fits',
    'jw02221002002_02201_00002_nrca1_cal.fits',
    'jw02221002002_02201_00001_nrca1_cal.fits',
    'jw02221002002_02201_00003_nrca1_cal.fits',
    'jw02221002002_02201_00004_nrca1_cal.fits',
    'jw02221002002_08201_00004_nrca4_cal.fits',
    'jw02221002002_08201_00003_nrca4_cal.fits',
    'jw02221002002_08201_00002_nrca4_cal.fits',
    'jw02221002002_08201_00001_nrca4_cal.fits',
    'jw02221002002_02201_00001_nrca2_cal.fits',
    'jw02221002002_02201_00002_nrca2_cal.fits',
    'jw02221002002_02201_00003_nrca2_cal.fits',
    'jw02221002002_02201_00004_nrca2_cal.fits',

    'jw02221002001_02201_00004_nrcb4_cal.fits',
    'jw02221002001_02201_00003_nrcb4_cal.fits',
    'jw02221002001_02201_00002_nrcb4_cal.fits',
    'jw02221002001_02201_00001_nrcb4_cal.fits',
    'jw02221002001_08201_00004_nrcb2_cal.fits',
    'jw02221002001_08201_00003_nrcb2_cal.fits',
    'jw02221002001_08201_00002_nrcb2_cal.fits',
    'jw02221002001_08201_00001_nrcb2_cal.fits',
    'jw02221002001_02201_00004_nrcb3_cal.fits',
    'jw02221002001_02201_00003_nrcb3_cal.fits',
    'jw02221002001_02201_00002_nrcb3_cal.fits',
    'jw02221002001_02201_00001_nrcb3_cal.fits',
    'jw02221002001_08201_00001_nrcb1_cal.fits',
    'jw02221002001_08201_00002_nrcb1_cal.fits',
    'jw02221002001_08201_00003_nrcb1_cal.fits',
    'jw02221002001_08201_00004_nrcb1_cal.fits',

    'jw02221002002_08201_00001_nrca1_cal.fits',
    'jw02221002002_08201_00002_nrca1_cal.fits',
    'jw02221002002_08201_00003_nrca1_cal.fits',
    'jw02221002002_08201_00004_nrca1_cal.fits',
    
    'jw02221002002_08201_00002_nrca2_cal.fits',
    'jw02221002002_08201_00001_nrca2_cal.fits',
    'jw02221002002_08201_00003_nrca2_cal.fits',
    'jw02221002002_08201_00004_nrca2_cal.fits',
    
    'jw02221002001_02201_00004_nrcb2_cal.fits',
    'jw02221002001_02201_00003_nrcb2_cal.fits',
    'jw02221002001_02201_00002_nrcb2_cal.fits',
    'jw02221002001_02201_00001_nrcb2_cal.fits',
    
    'jw02221002001_02201_00001_nrcb1_cal.fits',
    'jw02221002001_02201_00002_nrcb1_cal.fits',
    'jw02221002001_02201_00003_nrcb1_cal.fits',
    'jw02221002001_02201_00004_nrcb1_cal.fits',
]

custom_order_dict = {k: v + 1 for v, k in enumerate(custom_order)}

for m in mc:
    m.meta.group_id = str(custom_order_dict[m.meta.filename])

mc = ModelContainer(sorted(mc, key=lambda x: int(x.meta.group_id)))

In [6]:
# Run Image3 stage of the pipeline
im3 = Image3Pipeline()

im3.output_file = None
im3.output_ext = '.fits'
im3.output_use_model = False
im3.output_use_index = True
im3.save_results = True
im3.skip = False
im3.search_output_file = True

for s, step_pars in steps.items():
    step = getattr(im3, s)
    print(s, step)
    for a, v in step_pars.items():
        setattr(step, a, v)


im3.tweakreg.skip = False
# im3.tweakreg.use_custom_catalogs = True
im3.tweakreg.save_results=True
im3.source_catalog.skip = True

# im3.skymatch.save_results = True
im3.skymatch.skymethod = 'local'
im3.skymatch.subtract = False
# im3.skymatch.subtract = True
im3.skymatch.skystat = skystat
im3.skymatch.skip = False

im3.outlier_detection.skip=True

im3.resample.weight_type = weight_type
im3.resample.kernel = kernel

im3.run(mc)

2023-10-10 03:38:48,168 - stpipe.Image3Pipeline - INFO - Image3Pipeline instance created.
2023-10-10 03:38:48,168 - stpipe.Image3Pipeline.assign_mtwcs - INFO - AssignMTWcsStep instance created.
2023-10-10 03:38:48,169 - stpipe.Image3Pipeline.tweakreg - INFO - TweakRegStep instance created.
2023-10-10 03:38:48,170 - stpipe.Image3Pipeline.skymatch - INFO - SkyMatchStep instance created.
2023-10-10 03:38:48,170 - stpipe.Image3Pipeline.outlier_detection - INFO - OutlierDetectionStep instance created.
2023-10-10 03:38:48,171 - stpipe.Image3Pipeline.resample - INFO - ResampleStep instance created.
2023-10-10 03:38:48,172 - stpipe.Image3Pipeline.source_catalog - INFO - SourceCatalogStep instance created.
2023-10-10 03:38:48,354 - stpipe.Image3Pipeline - INFO - Step Image3Pipeline running with args (<ModelContainer>,).
2023-10-10 03:38:48,357 - stpipe.Image3Pipeline - INFO - Step Image3Pipeline parameters are: {'pre_hooks': [], 'post_hooks': [], 'output_file': None, 'output_dir': None, 'output

assign_mtwcs <jwst.assign_mtwcs.assign_mtwcs_step.AssignMTWcsStep object at 0x2b3be4d90>
tweakreg <jwst.tweakreg.tweakreg_step.TweakRegStep object at 0x2baf03a50>
skymatch <jwst.skymatch.skymatch_step.SkyMatchStep object at 0x2b3bc4410>
outlier_detection <jwst.outlier_detection.outlier_detection_step.OutlierDetectionStep object at 0x2bafa7290>
resample <jwst.resample.resample_step.ResampleStep object at 0x2bb06b3d0>
source_catalog <jwst.source_catalog.source_catalog_step.SourceCatalogStep object at 0x2c21a6610>


2023-10-10 03:38:48,791 - stpipe.Image3Pipeline - INFO - Prefetch for DRIZPARS reference file is '/Users/mcara/crds_cache/references/jwst/nircam/jwst_nircam_drizpars_0001.fits'.
2023-10-10 03:38:48,792 - stpipe.Image3Pipeline - INFO - Prefetching reference files for dataset: 'jw02221002001_02201_00002_nrca2_cal.fits' reftypes = ['drizpars']
2023-10-10 03:38:48,793 - stpipe.Image3Pipeline - INFO - Prefetch for DRIZPARS reference file is '/Users/mcara/crds_cache/references/jwst/nircam/jwst_nircam_drizpars_0001.fits'.
2023-10-10 03:38:48,793 - stpipe.Image3Pipeline - INFO - Prefetching reference files for dataset: 'jw02221002001_02201_00003_nrca2_cal.fits' reftypes = ['drizpars']
2023-10-10 03:38:48,794 - stpipe.Image3Pipeline - INFO - Prefetch for DRIZPARS reference file is '/Users/mcara/crds_cache/references/jwst/nircam/jwst_nircam_drizpars_0001.fits'.
2023-10-10 03:38:48,794 - stpipe.Image3Pipeline - INFO - Prefetching reference files for dataset: 'jw02221002001_02201_00004_nrca2_cal.f